In [3]:
from google.colab import drive
drive.mount('/content/drive')

# Cambia el path a donde tengas tus imágenes
data_dir = "/content/drive/MyDrive/Machine-learning"


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt

img_size = (224, 224)  # tamaño estándar para MobileNet/ResNet
batch_size = 32

train_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,   # 80% train, 20% valid
    subset="training",
    seed=123,
    image_size=img_size,
    batch_size=batch_size
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=img_size,
    batch_size=batch_size
)

# Mejorar rendimiento
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.prefetch(buffer_size=AUTOTUNE)


Found 1246 files belonging to 2 classes.
Using 997 files for training.
Found 1246 files belonging to 2 classes.
Using 249 files for validation.


In [5]:
base_model = tf.keras.applications.MobileNetV2(
    input_shape=img_size + (3,),
    include_top=False,   # quitamos la última capa (clasificación)
    weights="imagenet"
)

base_model.trainable = False  # congelamos las capas del modelo base


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


In [6]:
global_avg = layers.GlobalAveragePooling2D()(base_model.output)
output = layers.Dense(1, activation="sigmoid")(global_avg)

model = keras.Model(inputs=base_model.input, outputs=output)

model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    loss="binary_crossentropy",
    metrics=["accuracy"]
)


In [7]:
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=5
)


Epoch 1/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 116s 2s/step - accuracy: 0.8541 - loss: 0.3288 - val_accuracy: 0.9960 - val_loss: 0.0400
Epoch 2/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 67s 2s/step - accuracy: 0.9984 - loss: 0.0326 - val_accuracy: 1.0000 - val_loss: 0.0170
Epoch 3/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 81s 2s/step - accuracy: 0.9988 - loss: 0.0149 - val_accuracy: 1.0000 - val_loss: 0.0112
Epoch 4/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 82s 2s/step - accuracy: 1.0000 - loss: 0.0115 - val_accuracy: 1.0000 - val_loss: 0.0082
Epoch 5/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 82s 2s/step - accuracy: 1.0000 - loss: 0.0077 - val_accuracy: 1.0000 - val_loss: 0.0066


In [8]:
base_model.trainable = True
# entrenar solo las últimas capas
for layer in base_model.layers[:-50]:
    layer.trainable = False

model.compile(
    optimizer=keras.optimizers.Adam(1e-5),
    loss="binary_crossentropy",
    metrics=["accuracy"]
)

history_finetune = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=5
)


Epoch 1/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 108s 3s/step - accuracy: 0.8363 - loss: 0.3326 - val_accuracy: 0.9920 - val_loss: 0.0247
Epoch 2/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 81s 3s/step - accuracy: 0.9476 - loss: 0.1173 - val_accuracy: 0.9920 - val_loss: 0.0290
Epoch 3/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 82s 3s/step - accuracy: 0.9826 - loss: 0.0434 - val_accuracy: 0.9880 - val_loss: 0.0325
Epoch 4/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 81s 3s/step - accuracy: 1.0000 - loss: 0.0172 - val_accuracy: 0.9920 - val_loss: 0.0313
Epoch 5/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 83s 3s/step - accuracy: 1.0000 - loss: 0.0103 - val_accuracy: 0.9920 - val_loss: 0.0294


In [27]:
import numpy as np
from tensorflow.keras.preprocessing import image

img_path = "/content/drive/MyDrive/test/nube.jpg"  # ejemplo
img = image.load_img(img_path, target_size=img_size)
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = tf.keras.applications.mobilenet_v2.preprocess_input(x)

pred = model.predict(x)
print("Predicción:")
print(pred)
if(pred[0][0] > 0.65):
    print("Pizza 🍕")
else:
    print("Cielo ☁️")




1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
Predicción:
[[0.60736006]]
Cielo ☁️
